# NRP Tutorial: Baseball experiment

Welcome to this turorial. By following this notebook step by step, you will have fun and learn how to work with the Neurorobotics Platform.

The following notebook will guide you through setting up an experiment interactively, and perform data analysis and optimization offline afterwards. The experiment we will develop is inspired from the video with the baby learning to play ping pong.

In [ ]:
%autosave 0
from IPython.display import YouTubeVideo
YouTubeVideo("1n5uFGk2pnc")

# Getting Started

Please visit this tutorials's manualpage at https://developer.humanbrainproject.eu/docs/projects/HBP%20Neurorobotics%20Platform/1.2/nrp/tutorials/baseball/index.html

# Exercise 1: Setting up the experiment interactively

Start the NRP with the following commands:
```shell
cle-nginx
cle-start
cle-frontend
```
and clone and launch the experiment **Tutorial baseball experiment - Exercise**.

The experiment contains an iCub Robot in an empty world. Click the play button, nothing happens.
We will now implement the components one by one to have our iCub playing baseball controlled by a spiking neural network.

## Graphical settings - tune your rendering quality 

Open the **graphical settings** editor. In **Quality**, try setting antialiasing and shadows and see if this does not harm performance on your computer.
In **Environment**, test some skies and keep the one you like.

## Brain editor - Read how the brain is defined 

The PyNN script you read defines neuron populations.
In this experiment, the brain has 21 neurons and 2 populations. Population **sensors** defines sensory neurons, population **motors** motor neurons.

Open the **brain visualizer** to display your brain in 3D. Note that neuron positions are dynamically defined.

Try to change n_sensors to 5000, apply and see the changes in **brain visualizer**.
Change it back to 20 and apply.

## State Machine editor - Throwing balls

The first step is to have a ball throwing machine. Since this concerns purely the environment and not the robot, we implement this in a **state machine**.

Open the **NRP editors** (pen button in the toolbar), and select the **State Machine (SMACH)** tab.
Two **States** are defined: one to spawn a new ball, the other to remove the ball when it falls beyond a certain height.

Complete the code where you find the comments
```py
#################################################
# Insert code here:
#################################################
```

Don't forget to click the **Apply** button and **start** the experiment if necessary. Also, click the **Save** button to save your changes in the state machine. That way, your changes won't get lost if the experiment was stopped and relaunched.

## Transfer Functions editor - Moving the robot to a pre-swing pose - Neuron2Robot 

You should now have an experiment with balls that are being spawned and thrown in the direction of the iCub. We will first move the iCub to a proper swinging position. Since this concerns purely the robot, we implement this in a **transfer function (TF)**. 

Open the **NRP editors** again and select the **Transfer Functions** tab. Every function defined here is called at regular 20ms intervals.

Start with the function **simple_move_robot**. This is a **Neuron2Robot** TF but does not listen to any brain neuron - it simply sends motor commands to the robot in open loop.
Move the arm of the robot so that its hand hits the thrown balls.
When you're done, click the **Save** button to save your changes in the TF.

## Transfer Functions editor - Converting vision to brain activity - Robot2Neuron 

The robot has a brain of 20 sensor neurons and one motor neuron, connected all to all.
You can see the brain code in the **Brain Editor**. You can also visualize the neurons and their activity with the **brain visualizer** in the toolbar.

We want the camera images sensed by the robot to activate the sensor neurons. For this purpose, we will use a **Robot2Neuron** transfer function. Go to the function **grab_image** and activate the sensor neurons with respect to the difference in green values.
When you're done, click the **Save** button to save your changes in the TF.

You can see what the robot sees with the **generic image viewer** and the brain activity of a neuronal population with the **spiketrain**. The population displayed is defined in the **NeuronMonitor** transfer function and is set to the sensors population initially.

## Transfer Functions editor - Converting brain activity to action - Neuron2Robot 

The input neurons should now spike with respect to the positive difference of green in images.
We now want to make the robot arm swing with respect to the motor neuron activation. Implement the **Neuron2Robot** transfer function, here named **swing**.
When you're done, click the **Save** button to save your changes in the TF.

## Environment editor - build a nicer environment

Open the **environment editor** and select **Furniture**. Add a ping pong trainer robot and place it approximately at the origin of the ball's trajectory (you might have to move the trainer below ground).
Assemble enclosing walls using **Concrete Wall** and **Wood Wall** in **Construction**. You can use the **snap** option in the **object inspector** to move walls around and snap them to other wall parts.

## Stop your simulation

Click on the stop button to close the simulation. You do not need to save your changes since we will start from a fresh copy in the next exercise.


# Exercise 2: Offline analysis and optimization with virtual coach

In the first exercise, you interactively built your experiment so that it works with some hardcoded parameters.
In this exercise, we will run the experiment offline with the **Virtual Coach** to:
* Visualize some data
* Optimize some hardcoded parameters
You will see that the **Virtual Coach** has an API that replaces everything you did mannually in the first exercise: launch an experiment, start it, add a transfer function, set the brain,...

For this part of the notebook, you will need some extra libraries:
```
pip install --user pandas matplotlib
```

We will launch the same experiment you were modifying in Exercise 1.

In [ ]:
# disable global logging from the virtual coach
import logging
logging.disable(logging.INFO)
logging.getLogger('rospy').propagate = False
logging.getLogger('rosout').propagate = False

In [ ]:
# log into the virtual coach, update with your credentials
try:
    from hbp_nrp_virtual_coach.virtual_coach import VirtualCoach
    vc = VirtualCoach(environment='local', storage_username='nrpuser')
except ImportError as e:
    print(e)
    print("You have to start this notebook with the command:\
          cle-virtual-coach jupyter notebook")
    raise e

## First steps with the Virtual Coach

The virtual coach launch experiments return a well-documented simulation object with which you can interact:

In [ ]:
sim = vc.launch_experiment('tutorial_baseball_exercise_0')

In [ ]:
methods = [method for method in dir(sim) if not method.startswith('_')]
print('Available method on the simulation object:\n{}'.format('\n'.join(methods)))

In [ ]:
sim.register_status_callback?

If you have the frontend running, you can see that there is a running instance of this experiment if you click on it.
You can stop the experiment either with the frontend or with the virtual coach:

In [ ]:
sim.stop()

## Visualization and optimization

In the coming cells, you will have to use the virtual coach to optimize a metric by running the expriments multiple times.

Specifically, we will try to find the best synaptic weight between the input and the output populations.
The objective we try to maximize is the absolute **x** position of the ball (the ball is fired in the negative **x** direction, c.f. the StateMachine).

For this purpose we need:
1. the brain to be parametrizable
2. a way to access data from the experiment (the ball position)

### The brain

Here is the brain file from the **tutorial_baseball_exercise**. Note that the synaptic weight is replaced with the template parameter **{syn_weight}**:

In [ ]:
brain_template = '''
# -*- coding: utf-8 -*-
"""
Tutorial brain for the baseball experiment
"""

# pragma: no cover
__author__ = 'Jacques Kaiser'

from hbp_nrp_cle.brainsim import simulator as sim
import numpy as np

n_sensors = 20
n_motors = 1

sensors = sim.Population(n_sensors, cellclass=sim.IF_curr_exp())
motors = sim.Population(n_motors, cellclass=sim.IF_curr_exp())
sim.Projection(sensors, motors, sim.AllToAllConnector(),
               sim.StaticSynapse(weight={syn_weight}))
'''

### The (additional) transfer function

To record data from the experiment, we add a special type of transfer functions: **MapCSVRecorder**..
You could also add this transfer functions as a python file in the cloned experiment's directory **$HOME/.opt/nrpStorage/tutorial_baseball_exercise_0** and reference it in the **.bibi**, the results would be identical.

In [ ]:
record_ball_tf = \
'''
# Imported Python Transfer Function
import numpy as np
import sensor_msgs.msg

@nrp.MapCSVRecorder("ball_recorder", filename="ball_position.csv",
                    headers=["Time", "px", "py", "pz"])
@nrp.Robot2Neuron()
def record_ball_csv(t, ball_recorder):
    from rospy import ServiceProxy
    from gazebo_msgs.srv import GetModelState

    model_name = 'ball'
    state_proxy = ServiceProxy('/gazebo/get_model_state',
                                    GetModelState, persistent=False)
    ball_state = state_proxy(model_name, "world")

    if ball_state.success:
        current_position = ball_state.pose.position
        ball_recorder.record_entry(t,
                                   current_position.x, 
                                   current_position.y, 
                                   current_position.z)
'''

## Running the experiment with default synaptic weight

Run the experiment with the default synaptic weight, and save the resulting CSV file

In [ ]:
sim.get_csv_data?

In [ ]:
import csv
import tempfile
import time
import os

# this name has to match the name passed in the CSV transfer function
csv_name = 'ball_position.csv'
    
def save_position_csv(sim, datadir):
    with open(os.path.join(datadir, csv_name), 'wb') as f:
        cf = csv.writer(f)
        #################################################
        # Insert code here:
        # get the CSV data from the simulation
        #################################################
        cf.writerows(csv_data)
    
# The function make_on_status() returns a on_status() function
# This is called a "closure": 
# it is here used to pass the sim and datadir objects to on_status()
def make_on_status(sim, datadir):
    def on_status(msg):
        print("Current simulation time: {}".format(msg['simulationTime']))
        if msg['simulationTime'] == 5.0 and sim.get_state() == 'started':
            #################################################
            # Insert code here:
            # 1) pause the simulation, 
            # 2) save the CSV file
            # 3) stop the simulation
            #################################################
            print("Trial terminated - saved CSV in {}".format(datadir))
            
    return on_status
        
def run_experiment(datadir, brain_params={'syn_weight': 1.0}):
    #################################################
    # Insert code here:
    # 1) launch the experiment
    # 2) add the status callback
    # 3) add the parametrized brain file
    # 4) add the extra CSV TF
    # 5) start the simulation
    #################################################
    brain_file = brain_template.format(**brain_params)
    
    return sim
    
tmp_folder = tempfile.mkdtemp()
sim=run_experiment(datadir=tmp_folder)

If everything went fine, the csv data got saved to a **tempdir** in your **/tmp/**:

In [ ]:
csv_file = os.path.join(tmp_folder, csv_name)
print("Recorded the following csv file: {}".format(csv_file))

## Plotting the CSV data

Let's use [pandas](http://pandas.pydata.org/) to read the csv file, create dataframes and plot it.
You will need to install pandas:

```bash
pip install pandas
```

Let's read the csv file with pandas:

In [ ]:
import pandas
ball_csv = pandas.read_csv(csv_file)[['Time', 'px', 'py', 'pz']]
ball_csv

In [ ]:
%matplotlib inline
ball_csv=ball_csv.set_index('Time')
ball_csv.plot()

You can see that ball position is initially high when the ball is created, it then decreases until it hits the robot.

## Optimizing the synaptic weight

We define the objective function as being the accumulated **x** position. Let's run many trials with different synaptic weights and see the evolution of accumulated **x** position. For example, with this current trial of synaptic weight 1.0, the accumulated **x** is:

In [ ]:
ball_csv.px.sum()

Run the experiment again with different synaptic weights and plot the results.

In [ ]:
import numpy as np
n_trials = 2
trial_weights = np.linspace(0., 1.5, n_trials)
trial_ball_csv = [tempfile.mkdtemp() for i in range(n_trials)]
#################################################
# Insert code here:
# 1) run the experiments with all the trial_weights
# 2) compute the fitness of each trial
# 3) plot your results
#################################################


## Going further

In this second exercise, you saw how to run the same experiment many times to optimize a single synaptic weight.
Despite that the experiment was simple, you can keep the same workflow for your use case:
* Try to optimize different metric by recording different data in your CSV
* Try to optimize different parameters (e.g., many synaptic weights or learning rules hyperparameters)
* Try to use a different optimization algorithm (evolution strategy, ...)